Facebook Social Media API

In [9]:
import os
import json
import urllib
import pprint

In [10]:
# Since the code output in this notebook leaks the app_secret,
# it has been reset by the time you read this.

ACCESS_TOKEN = "EAACEdEose0cBAFEuZBA3TZA5dx736pgGbqiGZB2h6DOq5v7tOXqnLoivTytl2i0UpCjgC0LBK7yPLZCaK4w98TPVt84IFj6k9QZCpV3LBGLMEVJj6W8Fmu0zAY1OOolQYjYC4gcosyBERrGpUSSJbRUy3uPy4xEZCxjsNEekykICUsfFeJx9v4Bd311VJdrxcZD"

In [11]:
page_id = 'kompasCOM'

In [17]:

def testFacebookPageData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.12"
    node = "/" + page_id
    parameters = "/?access_token=%s" % ACCESS_TOKEN
    url = base + node + parameters
    
    # retrieve data
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    data = json.loads(response.read())
    
    print(json.dumps(data, indent=4, sort_keys=True))
    

testFacebookPageData(page_id, ACCESS_TOKEN)

{
    "id": "17353926534",
    "name": "Kompas.com"
}


In [19]:
def request_until_succeed(url):
    req = urllib.request.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib.request.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception as e:
            print(e)
            time.sleep(5)
            
            print("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return response.read()

In [20]:
def testFacebookPageFeedData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.12"
    node = "/" + page_id + "/feed" # changed
    parameters = "/?access_token=%s" % ACCESS_TOKEN
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    print(json.dumps(data, indent=4, sort_keys=True))
    

testFacebookPageFeedData(page_id, ACCESS_TOKEN)

{
    "data": [
        {
            "created_time": "2018-04-02T01:31:29+0000",
            "id": "17353926534_10156606654416535",
            "message": "Tragedi Kebakaran Mal Tewaskan 64 Orang, Gubernur di Rusia Mundur"
        },
        {
            "created_time": "2018-04-02T01:20:29+0000",
            "id": "17353926534_10156606629511535",
            "message": "Sandiaga Minta Pelajar yang Akan Ujian Nasional Tak Contoh Dirinya"
        },
        {
            "created_time": "2018-04-02T00:59:19+0000",
            "id": "17353926534_10156606579686535",
            "message": "Cheryl Cole Angkat Bicara Soal Kabar Perselingkungan Liam Payne"
        },
        {
            "created_time": "2018-04-02T00:38:14+0000",
            "id": "17353926534_10156606521126535",
            "message": "Huawei P20 Edisi Porsche Dijual Rp 35 Juta"
        },
        {
            "created_time": "2018-04-02T00:23:33+0000",
            "id": "17353926534_10156606477656535",
            "me

In [23]:
def getFacebookPageFeedData(page_id, ACCESS_TOKEN, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, ACCESS_TOKEN) # changed
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data
    

test_status = getFacebookPageFeedData(page_id, ACCESS_TOKEN, 1)["data"][0]
print(json.dumps(test_status, indent=4, sort_keys=True))

{
    "comments": {
        "data": [
            {
                "created_time": "2018-04-02T01:35:03+0000",
                "id": "10156606654416535_10156606661896535",
                "message": "Sikap yg terpuji . Salut \ud83d\udc4d\ud83d\udc4d\ud83d\udc4f\ud83d\udc4f"
            }
        ],
        "paging": {
            "cursors": {
                "after": "NAZDZD",
                "before": "NAZDZD"
            },
            "next": "https://graph.facebook.com/v2.5/17353926534_10156606654416535/comments?access_token=EAACEdEose0cBAFEuZBA3TZA5dx736pgGbqiGZB2h6DOq5v7tOXqnLoivTytl2i0UpCjgC0LBK7yPLZCaK4w98TPVt84IFj6k9QZCpV3LBGLMEVJj6W8Fmu0zAY1OOolQYjYC4gcosyBERrGpUSSJbRUy3uPy4xEZCxjsNEekykICUsfFeJx9v4Bd311VJdrxcZD&summary=true&limit=1&after=NAZDZD"
        },
        "summary": {
            "can_comment": true,
            "order": "ranked",
            "total_count": 4
        }
    },
    "created_time": "2018-04-02T01:31:29+0000",
    "id": "17353926534_10156606654416535",

In [22]:
def processFacebookPageFeedStatus(status):
    
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
    
    # Additionally, some items may not always exist,
    # so must check for existence first
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
    
    
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    # return a tuple of all processed data
    return (status_id, status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares)

processed_test_status = processFacebookPageFeedStatus(test_status)
print(processed_test_status)

NameError: name 'datetime' is not defined